In [1]:
%cd C:/Users/gioc4/Documents/python-benchmark

C:\Users\gioc4\Documents\python-benchmark


In [2]:
import pandas as pd
from provider_scorecard.benchmark import Benchmark

# load data
df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")
 
# transform
# get only the top 30 drg codes
df = df[df['Hospital County'] == 'Suffolk']

# some recoding
df['Length of Stay'] = [120 if x == '120 +' else x for x in df['Length of Stay']]
df['Length of Stay'] = df['Length of Stay'].astype(int)
df['APR DRG Code'] = df['APR DRG Code'].astype(str)

# top 30 drg codes
drg = df.groupby('APR DRG Code')['APR DRG Code'].count().sort_values(ascending=False).head(30)
drg = drg.astype(str)

# final dataframe
df = df[df['APR DRG Code'].isin(drg.index)]

C:\Users\gioc4\AppData\Local\Temp\ipykernel_10768\612499255.py:5: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")


In [3]:
df['Facility Name'].unique()

array(['Long Island Community Hospital', 'Peconic Bay Medical Center',
       'South Shore University Hospital', 'Huntington Hospital',
       'Stony Brook University Hospital',
       'Stony Brook Southampton Hospital',
       'Stony Brook Eastern Long Island Hospital',
       'Good Samaritan Hospital Medical Center', 'St. Charles Hospital',
       'St Catherine of Siena Hospital',
       'John T Mather Memorial Hospital of Port Jefferson New York Inc'],
      dtype=object)

In [4]:
df.groupby('Facility Name')['Facility Name'].count().sort_values(ascending=False)

Facility Name
Stony Brook University Hospital                                   17831
Good Samaritan Hospital Medical Center                            13667
South Shore University Hospital                                   12709
Huntington Hospital                                               10601
St. Charles Hospital                                               7790
Peconic Bay Medical Center                                         5468
St Catherine of Siena Hospital                                     5314
Long Island Community Hospital                                     4574
John T Mather Memorial Hospital of Port Jefferson New York Inc     4178
Stony Brook Southampton Hospital                                   3057
Stony Brook Eastern Long Island Hospital                            847
Name: Facility Name, dtype: int64

In [5]:
# input features
xfeat = [
    "Age Group",
    "Gender",
    "Race",
    "Length of Stay",
    "Type of Admission",
    "APR DRG Code",
    "APR Severity of Illness Description",
    "APR Risk of Mortality",
]

# evaluation features
efeat = ["Total Charges", "Total Costs", "Payment Typology 1"]

# focal hospitals
hosp = [
    'Stony Brook University Hospital',
    'Good Samaritan Hospital Medical Center',
    'South Shore University Hospital',
    'Huntington Hospital',
    'St. Charles Hospital',
    'Long Island Community Hospital'
]

In [6]:
res_list = []
mod_list = []

# loop through hospitals, store dataframes in list
for h in hosp:
    tr = df["Facility Name"] == h

    bch = Benchmark(
        data=df, focal_indicator=tr, predictor_features=xfeat, evaluation_features=efeat
    )
    bch.fit()
    bch.evaluate()

    # store results
    mod_list.append(bch)
    res_list.append(
        pd.DataFrame(
            {"Hospital": h, "Outcome": bch.Xeval.columns, "Value": bch.outcomes}
        )
    )

In [7]:
# concat into single pandas dataframe
results = pd.concat(res_list, axis=0)

In [18]:
mod_list[5].calc_balance()

Length of Stay:(6.65, 6.55)
Age Group_0 to 17:(0.0, 0.0)
Age Group_18 to 29:(0.03, 0.03)
Age Group_30 to 49:(0.13, 0.13)
Age Group_50 to 69:(0.38, 0.38)
Age Group_70 or Older:(0.45, 0.46)
Gender_F:(0.49, 0.49)
Gender_M:(0.51, 0.51)
Race_Black/African American:(0.11, 0.1)
Race_Multi-racial:(0.0, 0.0)
Race_Other Race:(0.12, 0.12)
Race_White:(0.78, 0.78)
Type of Admission_Elective:(0.03, 0.03)
Type of Admission_Emergency:(0.97, 0.97)
Type of Admission_Newborn:(0.0, 0.0)
Type of Admission_Not Available:(0.0, 0.0)
Type of Admission_Trauma:(0.0, 0.0)
Type of Admission_Urgent:(0.0, 0.0)
APR DRG Code_137:(0.09, 0.09)
APR DRG Code_139:(0.04, 0.04)
APR DRG Code_140:(0.04, 0.04)
APR DRG Code_174:(0.03, 0.03)
APR DRG Code_175:(0.01, 0.01)
APR DRG Code_192:(0.01, 0.01)
APR DRG Code_194:(0.07, 0.07)
APR DRG Code_201:(0.03, 0.03)
APR DRG Code_244:(0.01, 0.02)
APR DRG Code_249:(0.01, 0.01)
APR DRG Code_254:(0.02, 0.02)
APR DRG Code_263:(0.02, 0.02)
APR DRG Code_308:(0.02, 0.02)
APR DRG Code_324:(0.01,

In [13]:
res_list[0]

,Hospital,Outcome,Value
0,Stony Brook University Hospital,Total Charges,6048.00
1,Stony Brook University Hospital,Total Costs,5338.00
2,Stony Brook University Hospital,Payment Typology 1_Blue Cross/Blue Shield,0.02
3,Stony Brook University Hospital,Payment Typology 1_Department of Corrections,-0.00
4,Stony Brook University Hospital,Payment Typology 1_Federal/State/Local/VA,-0.00
5,Stony Brook University Hospital,"Payment Typology 1_Managed Care, Unspecified",-0.08
6,Stony Brook University Hospital,Payment Typology 1_Medicaid,0.02
7,Stony Brook University Hospital,Payment Typology 1_Medicare,-0.03
8,Stony Brook University Hospital,Payment Typology 1_Miscellaneous/Other,0.00
9,Stony Brook University Hospital,Payment Typology 1_Private Health Insurance,0.01
